# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import math
from nltk.tokenize import sent_tokenize


from IPython.display import Markdown, display

In [2]:
import sys
sys.path.append('..')

In [3]:
# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

In [4]:
# import gzip
# import math
# import random
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.ensemble import GradientBoostingRegressor, \
# GradientBoostingClassifier, RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import SGDClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix, \
# classification_report, make_scorer
# import statsmodels.api as sm

In [5]:
# From this project
# from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie

In [6]:
%reload_ext autoreload
from subjective_filter import SubjectiveFilter

## Configuration

In [7]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'


In [8]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

In [9]:
SAMPLE_FRACTION = 0.4

test_dic = {'train': {}, 'test':{}}

for i in ['train','test']:
    for j in ['positive','negative']:
         test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .drop(['reviewerName', 'helpful',
                   'summary', 'unixReviewTime', 
                   'reviewTime'], axis=1)


In [10]:
test_dic['train']['positive']

,asin,reviewerID,reviewText,overall
0,0,A32244V7CQUBD6,This video actually focuses mostly on one of the characters that Emmanuelle (Krista Allen) is trying to teach about sex & love. It's still pretty entertaining but if you are mostly interested in Kirsta Allen then you should know that she's not really in much of this episode.,4.0
1,1,A32244V7CQUBD6,"This episode pretty much has Hafron and Emmanuelle teleporting to different parts of the world and &quot;doing it&quot;. There is the continuing plot from an earlier episode of some group on Earth trying to track them down. That's the main reason for Emmanuelle and Hafron to jump to different parts of the world. Otherwise, this episode is mostly sex scenes.",4.0
2,2,A33KKMGGVLZ29T,"This is an intimate concert of Robert Mirabal. Although I thought that it was, as I said, masterful, the sound, at times sounded a little muffled.The storytelling of the songs gave an insight of native culture and of Mirabal's own family stories and history.The Dance and Ee You Oo are my picks for the best songs, but they are all a joy to watch. The Rare Tribal Mob and the Mirabal Singers/Dancers are great and provide a mesmerising stage performance.Very enjoyable",4.0
3,3,A33P47VEH0YULL,"A well put together DVD for the ""Stinkiest band"" in the world. Easy to navigate, and with some pretty interesting side notes...a great collectors item for any fan of Cradle of Filth or Swedish metal.",4.0
4,4,ADG33WELAQRZJ,"Great show from the inventors of the extreme gothic metal genre !! 75'of pure mayhem with good visuals and a great sound, although the vocals are a little high in the mix for my taste and the guitar on the right channel is 10 times louder than the one on the left so I ended up switching it to mono to even things up. It's hard to give this more stars esp. when you've watched the PTSFirepower DVD first like I did, that one is so much better. The 5 bonus videoclips totally rule and round up the package making the purchase a worthy one.Now the mockumentary may not be suited to everyone's tastes, personally it bored me and know of a lot of people who feels it ruins the band's image. Well don't watch it then.",4.0
5,5,A3YXITJWFW4BW,"I rented this dvd, i didnt buy it. I also rented PanDamonAeon and Mannequin. This dvd has excellent very long concert footage. Its practically the same songs as on Livebait for the Dead. And then we have the rest of the dvd. Im sorry, i love CoF but that ""schockumentary"" thing was just sad. I could barely stand to watch it. It's stupid and boring, maybe if theyd shown more of the band in a more coherent manner...and it just goes on and on forever. This is a 2 hour plus dvd adn half of it is crap. The ""blair twit Project"" is slightly amusing, but mostly boring. The music videos are good though. I would still recommend buying this dvd just for the music videos and the concert footage. PanDamonAeon gave me more joy though.",4.0
6,6,A214NHULS3H7OX,"Excellent Sound, Excellent Picture, Tons and Tons of songs. Love the Scorched Earth Erotic video. The only reason I didn't give this 5 stars is because I don't think they let their female singer sing enough, it adds a nice goth touch. I also did not give it five stars because it would have been nice to have the ""her ghost in the fog"" video on here, and finally I thin Dani could have done better on some parts of the songs. For example: On ""her ghost in the fog"" (live version), he chooses to growl in that typical death metal voice, instead of talking in the deep gothic voice like he does on the album. By using this growly voice, I think he is cheese'n off some of the songs best parts. Anyway, these are minor gripes and this is a must have DVD, so definitely BUY IT!...",4.0
7,7,A2IZOU2G1QX0JD,"As most of my friends hate, I'm massively into Cradle of Filth. In the style of Napoleon Dynamite describing a Liger, I would say ""...they're pretty much my favorite band."" Their music is among the worst in the industry, they suck in a 

In [11]:
total = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        print(test_dic[i][j].shape)
        total += test_dic[i][j].shape[0]
total

(57600, 4)
(57600, 4)
(14400, 4)
(14400, 4)


144000

In [ ]:
# pickle_in = open(data_path
#                 + 'movie_reviews_144000Pos_Neg_Samples.pkl'
#                 , "rb")
# test_dic = pickle.load(pickle_in)

## Remove objective sentences for case B

In [12]:
OBJ_THRESHOLD = 0.8

In [13]:
%reload_ext autoreload
obj_path = '../obj_subj_dev/'
fit_obj_tf = obj_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl'
fit_obj_model = obj_path + 'GBC_300_0.5_5_0.88cv.pkl'
subj_filter = SubjectiveFilter(fit_obj_tf, fit_obj_model)

In [14]:
# for tt in ['train','test']:
#     for pn in ['positive','negative']:
#         print(tt,pn,test_dic[tt][np].shape)
total = 0
for tt in test_dic.values():
    for df in tt.values():
        total += df.shape[0]
        display(df.shape)
print(total)

(57600, 4)

(57600, 4)

(14400, 4)

(14400, 4)

144000


In [ ]:
# test_dic['train']['positive'].iloc[40162:40200,:]

In [ ]:
# from IPython.display import Markdown, display

# for tt in test_dic.values():
#     for df in tt.values():
#         display(df.head(3))

In [ ]:
# type(test_dic['train']['positive']['reviewText'])

In [ ]:
# test = test_dic['train']['positive'].drop(['sentence','words'],axis=1).iloc[:2,:]  # 
# test

In [ ]:
# res = subj_filter.transform(
#             test,
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
# tp = subj_filter.transform(
#             test_dic['train']['positive'].drop(['sentence','words'],axis=1,errors='ignore'),
#             'reviewText', 
#             threshold=OBJ_THRESHOLD,
#             debug_level=2)

In [ ]:
# pickle_out = open(data_path + 'tp_subj_movie_reviews_0.8.pkl', "wb")
# pickle.dump(movie_reviews, pickle_out)

In [ ]:
# df.iloc[:5,:]

In [15]:
DROP_RATIO = 0.15
CHUNK_SZ = 5000

sent_dfs = {'train':{},'test':{}}

for ttname, tt in test_dic.items():
    for pn, df in tt.items():
        df = df.drop(['sentence','words'],axis=1,errors='ignore')
        df_list = []
        start = 0
        while start < df.shape[0]:
            end = start + CHUNK_SZ
            df2 = subj_filter.to_one_sent_per_row(
                    df.iloc[start:end,:]) \
                .rename(columns={"sentence": "reviewText"})
            df_list.append(
                subj_filter.transform(
                    df2,
                    'reviewText', 
                    threshold = None,
                    debug_level=0,
                    switch=True,
                    drop_ratio=DROP_RATIO))  
            
            start = end

        sent_dfs[ttname][pn] = df_list.pop()
        while len(df_list) > 0:
            sent_dfs[ttname][pn] = pd.merge(df_list.pop(), 
                                            sent_dfs[ttname][pn], how='outer')
#         res[ttname][pn] = reduce(lambda x, y: pd.merge(x, y, on = ['reviewerID','asin']), dfList)


../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


#### => Removed 5957 (15%) objective sentences

../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


#### => Removed 5706 (15%) objective sentences

../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


...

#### => Removed 4967 (15%) objective sentences

../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


#### => Removed 5542 (15%) objective sentences

../subjective_filter.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['sentence'] = df['reviewText'].map(sent_tokenize)


#### => Removed 4635 (15%) objective sentences

In [16]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

/anaconda3/lib/python3.7/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,reviewerID,asin,reviewText,overall


## Save / Load

In [ ]:
# pickle_out = open(data_path + 'sent_dfs_144k.pkl', "wb")
# pickle.dump(sent_dfs, pickle_out)

In [ ]:
# pickle_in = open(data_path
#                 + 'sent_dfs_144k.pkl'
#                 , "rb")
# sent_dfs = pickle.load(pickle_in)

In [ ]:
for ttname, tt in sent_dfs.items():
    for pn, df in tt.items():
        display(df.head(3))

In [19]:
df1 = sent_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]
df1

,reviewerID,asin,reviewText,overall
0,A09409663AZ10ZKCJTTL7,959,My husband liked it very much. The CLEAN ONES He loves Mom's Mabley .,4.0
1,A0999509ZRK5LA9PJ22N,811,well written story and well acted by players that you know and enjoy. i think this movie should have gotten better reviews when it first came out.,4.0
2,A100JCBNALJFAW,4570,"wow this movie blew me away, i saw it with a mind set that i was gonna hate but i didnt it was really good, and it had some comedy in it as well.now there are a few part that are a little cheap but its a fine damn movie. besides what other movie has a giant bunny?",4.0
3,A102VX16UZAM85,2443,"If you are very overweight or outta shape YOU MAY NOT WANT TO START HERE. That being said if you stick with this workout you will see results in 10 workouts or less as stated on the box. This is the workout that introduced me to the ""Firm"" and it is tough! The older Firm tapes are pretty good. This is one of the old Firm tapes made from clips of older Firm workouts.",4.0
4,A102ZVRQ4ZTX5Q,2187,"So this time JW make sure it's all top notch quality. I certainly hope he can get the Mid Nite Special &amp; other tv show film of him, as i'd personally like tosee it all, especially any Johnny Winter And footage. Yep, the other reviewers were right when speaking of the qualityof many of the performances on here. STEVEN MUNARI Since on Johnny's website(just use his name to get there)they are asking for anyone withfilm footage or more recent video footage to send it in, I guessJW will do another dvd? The quality is poor, but i'll take any footage of JW possible.",4.0
5,A103MRWENLU3IG,1069,"Following the failure of just about everything, from poor intelligence to lack of communication this movie shows exactly how rough it was to serve in the military during the 1980's. The gun fights are nothing to winch at but, lack the extreme effects and graphic (to date) to make you feel you are there. If only it could have a bigger budget to expand on some areas and added little touches. The movie follows the book very closely, which makes for a very great entertainment. An Grade A example of why the military spends so much money on equipment today. This movie is a must for any fan that is looking for a Sean Bean collection. Which makes it a must for any historian or military ""buff"". Very well written both in book and movie. The locations are very well scouted and whole military attitude is very accurate. Sean Bean is great as both the lead and main character. A must watch for the escape and evasion of Britain's finest military professionals alone. The actors display a grace under fire type of attitude even during capture that is for the military personnel to exemplify. Gripping, exciting filled with real life type of situation.",4.0
6,A106016KSI0YQ,3898,"The screenwriters give you so many fine red herrings that the ending works quite well. Peter Mullan gives a superb performance as the member of the hazmat team who most engages the audience's sympathies (though his Scots accent is at times a bit impenetrable). This is the standard set-up for a Gothic film (think THE SHINING, the orginal HAUNTING, the first ALIEN), but the unusualness of the hospital, the exceptionally fine performances, and the sophisticated script set this in nearly as high the same class as those aforementioned Gothic classics. The cinematography is exceptionally and beautifully creepy. The actual abandoned Danvers State Hospital in Massachusetts is a fantastic Gothic revival pile of a building; it is mostrously oversized, larger than you'd think any such building would have a right to be, and makes an absolutely wonderful setting for this very sophisticated Gothic chiller. Four workers on a hazmat team have one week to remove the asbestos and other hazardous waste products from the abandoned state hospital, with its miles of ugly rotting corridors and empty rooms; by the end of the film there are fewer of the team than when we started.",4.0
7,A1080S66TVH9C

## Remove obj sentences

In [17]:
subj_dfs = sent_dfs

In [ ]:
# Crashes kernel
# DROP_RATIO = 0.15

# subj_dfs = {'train':{},'test':{}}

# for ttname, tt in sent_dfs.items():
#     for pn, df in tt.items():
#            subj_dfs[ttname][pn] = subj_filter.transform(
#                     df,
#                     'reviewText', 
#                     threshold = None,
#                     debug_level=0,
#                     switch=True,
#                     drop_ratio=DROP_RATIO)

In [ ]:
# %reload_ext autoreload
# from subjective_filter import SubjectiveFilter

# DROP_RATIO = 0.15
# CHUNK_SZ = 50000
# subj_dfs = {'train':{},'test':{}}

# for ttname, tt in sent_dfs.items():
#     for pn, df in tt.items():
#         df_list = []
#         start = 0
#         while start < df.shape[0]:
#             end = start + CHUNK_SZ
#             df_list.append(
#                 subj_filter.transform(
#                     df.iloc[start:end,:],
#                     'reviewText', 
#                     threshold = None,
#                     debug_level=0,
#                     switch=True,
#                     drop_ratio=DROP_RATIO))      
#             start = end

#         subj_dfs[ttname][pn] = df_list.pop()
#         while len(df_list) > 0:
#             subj_dfs[ttname][pn] = pd.merge(df_list.pop(), 
#                                             subj_dfs[ttname][pn], how='outer')

In [ ]:
df1 = subj_dfs['train']['positive']
df1[(df1['reviewerID'] == 'A32244V7CQUBD6') & (df1['asin'] == 'B00005QFEK')]

In [ ]:
# TEST connection
# CHUNK_SZ = 5
# subj_dfs = {'train':{},'test':{}}
# df_test = pd.DataFrame({'A': [1,2,3,4,5,6], 'B':[1,2,3,4,5,6]})

# df_list = []
# start = 0
# while start < df_test.shape[0]:
#     end = start + CHUNK_SZ
#     df_list.append(df_test.iloc[start:end,:])
#     start = end

# out = df_list.pop()
# while len(df_list) > 0:
#     out = pd.merge(df_list.pop(), out, how='outer')
# out

In [ ]:
# for ttname, tt in sent_dfs.items():
#     for pn, df in tt.items():
#         subj_filter.transform(
#             df.rename(columns={"reviewText": "sentence"}),
#             'reviewText', 
#             threshold = None,
#             debug_level=0,
#             switch=True,
#             drop_ratio=0.15)
#         .rename(columns={"sentence": "reviewText"})

In [18]:
total = 0
old_total = 0
for tt in ['train', 'test']:
    for pn in ['positive', 'negative']:
        print('\n {} {}'.format(tt,pn))
        tot = test_dic[tt][pn].shape[0]
        new = subj_dfs[tt][pn].shape[0]
        dropped = tot - new
        print('Dropped: {0} ({1:.1%})'.format(dropped, dropped/tot))
        total += new
        old_total += tot
print('\nOld total:', old_total)
print('New total:', total)
print('Removed a total of:', old_total - total, 'reviews')


 train positive
Dropped: 407 (0.7%)

 train negative
Dropped: 381 (0.7%)

 test positive
Dropped: 91 (0.6%)

 test negative
Dropped: 81 (0.6%)

Old total: 144000
New total: 143040
Removed a total of: 960


In [24]:
subj_ids = set(subj_dfs['train']['positive']['asin'])
all_ids = set(test_dic['train']['positive']['asin'])
all_ids - subj_ids

{94,
 130,
 172,
 217,
 378,
 633,
 824,
 859,
 920,
 945,
 1077,
 1171,
 1199,
 1413,
 1422,
 1426,
 1917,
 1949,
 2307,
 2319,
 2544,
 2610,
 2707,
 2734,
 2895,
 3162,
 3180,
 3296,
 3384,
 3417,
 3610,
 3825,
 3865,
 4081,
 4083,
 4195,
 4226,
 4752,
 4970,
 5150,
 5249,
 5487,
 5896,
 6006,
 6070,
 6309,
 6326,
 6538,
 6591,
 6679,
 6834,
 6851,
 6995,
 7010,
 7129,
 7162,
 7306,
 7338,
 7469,
 7542,
 7654,
 7655,
 8092,
 8281,
 8306,
 8331,
 8431,
 8547,
 8862,
 8888,
 8891,
 8961,
 9264,
 9409,
 9527,
 9639,
 9732,
 9800,
 9997,
 10074,
 10421,
 10660,
 10679,
 11038,
 11180,
 11418,
 11442,
 11468,
 11802,
 11947,
 11998,
 12148,
 12283,
 12296,
 12646,
 13058,
 13156,
 13232,
 13275,
 13860,
 14026,
 14062,
 14086,
 14195,
 14239,
 14384,
 14706,
 14763,
 15105,
 15137,
 15441,
 15452,
 16187,
 16284,
 16530,
 16553,
 16737,
 16995,
 17189,
 17259,
 17540,
 17582,
 17616,
 17829,
 17949,
 17995,
 18106,
 18188,
 18343,
 18355,
 18387,
 18426,
 18545,
 18761,
 18763,
 18980,
 1

In [20]:
pickle_out = open(data_path
                    + 'reviews_wout_most_subj_' + str(DROP_RATIO) + '.pkl'
                    , "wb")
pickle.dump(subj_dfs, pickle_out)
pickle_out.close()

In [ ]:
# for ttname, tt in test_dic.items():
#     for pn, df in tt.items():
#         display(df.head(20))

In [ ]:
test_dic['train']['positive']

In [ ]:
subj_dfs['train']['positive']

In [ ]:
# for ttname, tt in subj_dfs.items():
#     for pn, df in tt.items():
#         display(df.head(20))